In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train = train.values
test = test.values

X = train[:,1:]
Y = train[:,0]

X = np.reshape(X, (X.shape[0], 28, 28))
X = np.expand_dims(X, axis=3)

Y = np.reshape(Y, (-1, 1))
encoder = LabelBinarizer().fit(Y)
Y = encoder.transform(Y)

test = np.reshape(test, (test.shape[0], 28, 28))
test = np.expand_dims(test, axis=3)

print(X.shape, Y.shape, test.shape)

X = X/255.0
test = test/255.0

(42000, 28, 28, 1) (42000, 10) (28000, 28, 28, 1)


In [14]:
def cnn_model(features, labels, mode):
    
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
    tf.cast(input_layer, tf.float32)
    # Convolution 1
    conv1 = tf.layers.conv2d(
            inputs=input_layer,
            filters=32,
            kernel_size=[5,5],
            padding='same',
            activation=tf.nn.relu)
    
    # Pooling 1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    
    # Convolution 2
    conv2 = tf.layers.conv2d(
            inputs=pool1,
            filters=64,
            kernel_size=[5,5],
            padding='same',
            activation=tf.nn.relu)
    
    # Pooling 2
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
    pool2_flat = tf.reshape(pool2, [-1, 7*7*64])
    
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode==tf.estimator.ModeKeys.TRAIN)
    
    logits = tf.layers.dense(inputs=dropout, units=10)
    
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=optimizer)
    elif mode == tf.estimator.ModeKeys.EVAL:
        predictions = tf.argmax(logits, axis=1)
        origin = tf.argmax(labels, axis=1)
        accuracy = tf.metrics.accuracy(labels=origin, predictions=predictions)
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=accuracy)
    else:
        predictions = tf.argmax(logits, axis=1)
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=0)

digit_recognizer = tf.estimator.Estimator(model_fn=cnn_model, model_dir="./tmp")
train_input_fn = tf.estimator.inputs.numpy_input_fn(
                 x={"x":X_train},
                 y=Y_train,
                 batch_size=50,
                 num_epochs=None,
                 shuffle=True)

digit_recognizer.train(input_fn=train_input_fn, steps=10000)

eval_input_fn = tf.estimator.inputs.numpy_input_fn(
                x={"x":X_test},
                y=Y_test,
                num_epochs=1,
                shuffle=False)
eval_results = digit_recognizer.evaluate(input_fn=eval_input_fn)
print(eval_results)
    

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {}


TypeError: Input 'filter' of 'Conv2D' Op has type float32 that does not match type float64 of argument 'input'.